In [40]:
from dataclasses import dataclass
import asyncio

from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.hash.selector import get_selector_from_name
from starknet_py.net.client_models import Call

In [14]:
AMM_ADDRESS = 0x047472e6755afc57ada9550b6a3ac93129cc4b5f98f51c73e0644d129fd208d9 # Carmine Options Amm address

# RPC setup
RPC_URL = 'https://starknet-mainnet.public.blastapi.io'
NET = FullNodeClient(RPC_URL)

In [51]:
def felt_to_str(felt):
    num_bytes = (felt.bit_length() + 7) // 8
    bytes_ = felt.to_bytes(num_bytes, "big")
    return bytes_.decode("utf-8")

In [18]:
async def function_call(address: int, selector: str, calldata: list[int | None]) -> list[int]:
    call = Call(
        to_addr=address,
        selector=get_selector_from_name(selector),
        calldata=calldata
    )
    res = await NET.call_contract(call)
    return res

async def function_call_from_block(address: int, selector: str, calldata: list[int], block_num: int) -> list[int]:
    call = Call(
        to_addr=address,
        selector=get_selector_from_name(selector),
        calldata=calldata
    )
    res = await NET.call_contract(call, block_number=block_num)
    return res

In [25]:
# Fetch list of lptokens
lptokens = await function_call(AMM_ADDRESS, 'get_all_lptoken_addresses', [])
lptokens = lptokens[1:] # First element is length of the array 

In [62]:
@dataclass
class Token:
    """
    Class representing single Starknet token.
    """
    
    decimals: int
    symbol: str
    address: int

    @classmethod
    async def from_address(cls, address: int) -> 'Token':
        decimals = await function_call(address, 'decimals', [])
        symbol = await function_call(address, 'symbol', [])

        return Token(
            decimals=decimals[0],
            symbol= felt_to_str(symbol[0]),
            address = address
        )


@dataclass
class PoolInfo:
    """
    Class representing Carmine Options AMM pool info.
    """
    quote_token_address: int
    base_token_address: int
    option_type: int
    address: int

    @classmethod
    async def from_address(cls, address: int) -> 'PoolInfo':
        """
        Constructs Pool info from the address of the lptoken.
        """
        res = await function_call(
            AMM_ADDRESS, 
            'get_pool_definition_from_lptoken_address', [address]
        )
        return PoolInfo(
            quote_token_address=res[0],
            base_token_address=res[1],
            option_type=res[2],
            address=address
        )
    

In [63]:
pool_infos = [ PoolInfo.from_address(i) for i in lptokens ]
pool_infos = await asyncio.gather(*pool_infos)

In [64]:
pool_infos

[PoolInfo(quote_token_address=2368576823837625528275935341135881659748932889268308403712618244410713532584, base_token_address=2087021424722619777119509474943472645767659996348769578120564519014510906823, option_type=0, address=3188589015573695152709924369945883242091047925178524437070103516325412109404),
 PoolInfo(quote_token_address=2368576823837625528275935341135881659748932889268308403712618244410713532584, base_token_address=2087021424722619777119509474943472645767659996348769578120564519014510906823, option_type=1, address=1991040978832064286557357805465536070417175868071594335617512007012013076825),
 PoolInfo(quote_token_address=2368576823837625528275935341135881659748932889268308403712618244410713532584, base_token_address=1806018566677800621296032626439935115720767031724401394291089442012247156652, option_type=0, address=1522519488290211190981877881039507445699769297762330502060609080954142963419),
 PoolInfo(quote_token_address=2368576823837625528275935341135881659748932889268

In [75]:
CUBIT_SHIFT = 2**64

for pool in pool_infos:
    # Fetch unlocked and value of position
    unlocked = await function_call(AMM_ADDRESS, 'get_unlocked_capital', [pool.address])

    value_of_position = await function_call(AMM_ADDRESS, 'get_value_of_pool_position', [pool.address])
    value_of_position = value_of_position[0] / CUBIT_SHIFT 

    base_token = await Token.from_address(pool.base_token_address)
    quote_token = await Token.from_address(pool.quote_token_address)

    if pool.option_type == 0: # Call options
        type_str = 'Call'
        capital_token = base_token.symbol

        pool_tvl = (unlocked[0] / 10**base_token.decimals) + value_of_position 
    else: # Put
        type_str = 'Put'
        capital_token = quote_token.symbol

        pool_tvl = (unlocked[0] / 10**quote_token.decimals) + value_of_position 


    print(f'Tvl of {base_token.symbol}{quote_token.symbol} {type_str} pool: {pool_tvl} {capital_token}')
    

Tvl of ETHUSDC Call pool: 18.958805144076358 ETH
Tvl of ETHUSDC Put pool: 53014.5649812814 USDC
Tvl of WBTCUSDC Call pool: 0.01933811 WBTC
Tvl of WBTCUSDC Put pool: 16922.259529858664 USDC
Tvl of ETHSTRK Call pool: 2.8978208306499327 ETH
Tvl of ETHSTRK Put pool: 88632.20888050368 STRK
Tvl of STRKUSDC Call pool: 229976.3073524318 STRK
Tvl of STRKUSDC Put pool: 73726.87115775663 USDC
